In [22]:
import pandas as pd
import numpy as np

## Example 1

In [7]:
# 1. 读取数据
df = pd.read_csv("TSLA.csv", parse_dates=["Date"])
#df = df.sort_values("date").reset_index(drop=True)
df.head(5)

,Date,Open,High,Low,Close,Adj Close,Volume
0,2010-06-29,3.800,5.000,3.508,4.778,4.778,93831500
1,2010-06-30,5.158,6.084,4.660,4.766,4.766,85935500
2,2010-07-01,5.000,5.184,4.054,4.392,4.392,41094000
3,2010-07-02,4.600,4.620,3.742,3.840,3.840,25699000
4,2010-07-06,4.000,4.000,3.166,3.222,3.222,34334500


In [9]:
df = df.sort_values("Date").reset_index(drop=True)

In [10]:
# 2. 计算收益率
df["ret"] = df["Close"].pct_change()

In [11]:
# 3. 计算 Momentum（12-1）
df["momentum"] = df["Close"].shift(21) / df["Close"].shift(252) - 1

In [12]:
# 4. 生成交易信号
df["signal"] = (df["momentum"] > 0).astype(int)

In [13]:
# 5. 策略收益
df["strategy_ret"] = df["signal"].shift(1) * df["ret"]

In [14]:
# 6. 累计净值
df["cum_market"] = (1 + df["ret"]).cumprod()
df["cum_strategy"] = (1 + df["strategy_ret"]).cumprod()

In [15]:
df.head(5)

,Date,Open,High,Low,Close,Adj Close,Volume,ret,momentum,signal,strategy_ret,cum_market,cum_strategy
0,2010-06-29,3.800,5.000,3.508,4.778,4.778,93831500,NaN,NaN,0,NaN,NaN,NaN
1,2010-06-30,5.158,6.084,4.660,4.766,4.766,85935500,-0.002512,NaN,0,-0.0,0.997488,1.0
2,2010-07-01,5.000,5.184,4.054,4.392,4.392,41094000,-0.078473,NaN,0,-0.0,0.919213,1.0
3,2010-07-02,4.600,4.620,3.742,3.840,3.840,25699000,-0.125683,NaN,0,-0.0,0.803684,1.0
4,2010-07-06,4.000,4.000,3.166,3.222,3.222,34334500,-0.160937,NaN,0,-0.0,0.674341,1.0


In [19]:
#风险指标计算: 年化收益 & 波动率
annual_ret = df["strategy_ret"].mean() * 252
annual_ret

np.float64(0.46134814045733413)

In [18]:
annual_vol = df["strategy_ret"].std() * np.sqrt(252)
annual_vol

np.float64(0.482620172540724)

In [21]:
#最大回撤
cum = df["cum_strategy"]
rolling_max = cum.cummax()
drawdown = (cum - rolling_max) / rolling_max
max_drawdown = drawdown.min()
max_drawdown

-0.6062653672123575

Built and backtested a momentum-based quantitative trading strategy using A-share daily data,
including signal construction, performance evaluation, and risk analysis (volatility, max drawdown).

## Example 2 

In [35]:
df = pd.read_csv("StockPriceDataset.csv", parse_dates=["Date"])

In [36]:
df.head(5)

,Date,Open,High,Low,Close,Adj Close,Volume,Ticker
0,2014-01-02,19.845715,19.893929,19.715000,19.754642,17.273226,234684800,AAPL
1,2014-01-03,19.745001,19.775000,19.301071,19.320715,16.893808,392467600,AAPL
2,2014-01-06,19.194643,19.528570,19.057142,19.426071,16.985926,412610800,AAPL
3,2014-01-07,19.440001,19.498571,19.211430,19.287144,16.864458,317209200,AAPL
4,2014-01-08,19.243214,19.484285,19.238930,19.409286,16.971254,258529600,AAPL


In [37]:
prices = df.pivot(index="Date", columns="Ticker", values="Close")
prices = prices.sort_index()

In [38]:
# 3. Compute Daily Returns
returns = prices.pct_change().dropna()

In [39]:
type(returns)

pandas.core.frame.DataFrame

In [40]:
# 4. Momentum Factor (60-day)
lookback = 60
momentum = prices.pct_change(lookback)

In [41]:
# 5. Monthly Rebalancing Dates
rebalance_dates = returns.resample("ME").last().index

In [45]:
# 6. Portfolio Construction & Backtest

current_weights = pd.Series(0.0, index=prices.columns)
portfolio_returns = []

for i in range(1, len(rebalance_dates)):
    start = rebalance_dates[i - 1]
    end = rebalance_dates[i]

    # 🔑 Align to last available trading day
    signal_date = momentum.index.asof(start)
    if signal_date is None:
        continue

    signal = momentum.loc[signal_date].dropna()
    if signal.empty:
        continue

    cutoff = signal.quantile(1 - top_quantile)
    selected = signal[signal >= cutoff].index

    new_weights = pd.Series(0.0, index=prices.columns)
    new_weights[selected] = 1.0 / len(selected)

    turnover = (new_weights - current_weights).abs().sum()
    cost = turnover * transaction_cost

    period_returns = returns.loc[start:end].iloc[1:]
    daily_port_ret = period_returns.dot(new_weights)
    daily_port_ret.iloc[0] -= cost

    portfolio_returns.append(daily_port_ret)
    current_weights = new_weights






In [46]:
portfolio_returns = pd.concat(portfolio_returns)

In [47]:
# 7. Net Asset Value (NAV)
nav = (1 + portfolio_returns).cumprod()


In [48]:
# 8. Risk Metrics
# 8.1 Annualized Return & Volatility
annual_return = nav.iloc[-1] ** (252 / len(nav)) - 1
annual_vol = portfolio_returns.std() * np.sqrt(252)


In [49]:
# 8.2 Maximum Drawdown
rolling_max = nav.cummax()
drawdown = (nav - rolling_max) / rolling_max
max_drawdown = drawdown.min()


In [50]:
# 8.3 Value at Risk (VaR)
# (a) Historical VaR (95%)
VaR_95_hist = np.percentile(portfolio_returns, 5)


In [51]:
# (b) Parametric (Normal) VaR
mu = portfolio_returns.mean()
sigma = portfolio_returns.std()
z_05 = -1.645

VaR_95_param = mu + z_05 * sigma


In [52]:
# 9. Summary Table
summary = pd.DataFrame({
    "Metric": [
        "Annualized Return",
        "Annualized Volatility",
        "Max Drawdown",
        "95% Historical VaR",
        "95% Parametric VaR"
    ],
    "Value": [
        annual_return,
        annual_vol,
        max_drawdown,
        VaR_95_hist,
        VaR_95_param
    ]
})

print(summary)



                  Metric     Value
0      Annualized Return  0.248798
1  Annualized Volatility  0.231975
2           Max Drawdown -0.298173
3     95% Historical VaR -0.022175
4     95% Parametric VaR -0.023050


10. What This Code Demonstrates (Interview-Worthy)

✔ Proper factor construction

✔ Explicit backtest logic (no black box)

✔ Transaction cost modeling

✔ Risk-aware evaluation (VaR + drawdown)

✔ Clean separation of signal, execution, and evaluation

## 3 

In [57]:
import pandas as pd
import numpy as np

# ==============================
# 1️⃣ Load Price Data
# ==============================
# CSV format: date,ticker,adj_close
df = pd.read_csv("StockPriceDataset.csv", parse_dates=["Date"])
prices = df.pivot(index="Date", columns="Ticker", values="Adj Close")
prices = prices.sort_index()

# ==============================
# 2️⃣ Compute Returns & Momentum
# ==============================
returns = prices.pct_change().dropna()
lookback_mom = 60
momentum = prices.pct_change(lookback_mom)

# ==============================
# 3️⃣ Define Backtest Parameters
# ==============================
top_quantile = 0.2
transaction_cost = 0.001  # 10 bps

rebalance_dates = returns.resample("ME").last().index  # month-end rebalancing

# ==============================
# 4️⃣ Backtest Loop
# ==============================
portfolio_returns = []
current_weights = pd.Series(0.0, index=prices.columns)

for i in range(1, len(rebalance_dates)):
    start = rebalance_dates[i - 1]
    end = rebalance_dates[i]

    # Align to last available trading day
    signal_date = momentum.index.asof(start)
    if signal_date is None:
        continue

    signal = momentum.loc[signal_date].dropna()
    if signal.empty:
        continue

    # Select top quantile
    cutoff = signal.quantile(1 - top_quantile)
    selected = signal[signal >= cutoff].index

    # Equal-weight portfolio
    new_weights = pd.Series(0.0, index=prices.columns)
    new_weights[selected] = 1.0 / len(selected)

    # Turnover & transaction cost
    turnover = (new_weights - current_weights).abs().sum()
    cost = turnover * transaction_cost

    # Daily returns during holding period
    period_returns = returns.loc[start:end].iloc[1:]
    daily_port_ret = period_returns.dot(new_weights)
    daily_port_ret.iloc[0] -= cost

    portfolio_returns.append(daily_port_ret)
    current_weights = new_weights

portfolio_returns = pd.concat(portfolio_returns)
nav = (1 + portfolio_returns).cumprod()

# ==============================
# 5️⃣ Risk Metrics
# ==============================
def annualized_volatility(r):
    return r.std() * np.sqrt(252)

def max_drawdown(nav):
    rolling_max = nav.cummax()
    drawdown = (nav - rolling_max) / rolling_max
    return drawdown.min()

def var_historical(r, alpha=0.05):
    return np.percentile(r, 100 * alpha)

def var_parametric(r, alpha=0.05):
    mu = r.mean()
    sigma = r.std()
    z = -1.645  # 95% confidence
    return mu + z * sigma

def var_ewma(r, lambda_=0.94, alpha=0.05):
    """
    EWMA (RiskMetrics) 1-day VaR
    Returns a Series of daily EWMA VaR
    """
    ewma_var = np.zeros(len(r))
    sigma2 = r.iloc[0] ** 2
    for t in range(1, len(r)):
        sigma2 = lambda_ * sigma2 + (1 - lambda_) * r.iloc[t-1] ** 2
        ewma_var[t] = -np.sqrt(sigma2) * 1.645
    return pd.Series(ewma_var, index=r.index)

# ==============================
# 6️⃣ Futures Volatility (log returns)
# ==============================
def compute_futures_volatility(prices, lookback=20):
    log_returns = np.log(prices / prices.shift(1)).dropna()
    rolling_vol = log_returns.rolling(window=lookback).std() * np.sqrt(252)
    return rolling_vol

def ewma_futures_volatility(log_returns, lambda_=0.94):
    ewma_vol = pd.DataFrame(index=log_returns.index, columns=log_returns.columns)
    for col in log_returns.columns:
        sigma2 = log_returns[col].iloc[0] ** 2
        series = []
        for r in log_returns[col]:
            sigma2 = lambda_ * sigma2 + (1 - lambda_) * r ** 2
            series.append(np.sqrt(sigma2))
        ewma_vol[col] = series
    return ewma_vol * np.sqrt(252)  # annualized

futures_log_returns = np.log(prices / prices.shift(1)).dropna()
rolling_vol = compute_futures_volatility(prices, lookback=20)
ewma_vol = ewma_futures_volatility(futures_log_returns)

# ==============================
# 7️⃣ Compile Risk Summary
# ==============================
risk_summary = pd.DataFrame({
    "Metric": ["Annualized Volatility", "Max Drawdown", "95% Historical VaR", 
               "95% Parametric VaR", "EWMA VaR (last day)"],
    "Value": [
        annualized_volatility(portfolio_returns),
        max_drawdown(nav),
        var_historical(portfolio_returns),
        var_parametric(portfolio_returns),
        var_ewma(portfolio_returns).iloc[-1]
    ]
})

print(risk_summary)

# ==============================
# 8️⃣ Save Results
# ==============================
#nav.to_csv("results/nav.csv")
#risk_summary.to_csv("results/risk_summary.csv")
#ewma_vol.to_csv("results/ewma_futures_vol.csv")


                  Metric     Value
0  Annualized Volatility  0.231284
1           Max Drawdown -0.296395
2     95% Historical VaR -0.021966
3     95% Parametric VaR -0.022918
4    EWMA VaR (last day) -0.012189


## 4 

In [ ]:
# Credit Scoring
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.3, random_state=42)
model = RandomForestClassifier(n_estimators=200, random_state=42)
model.fit(X_train, y_train)
y_pred_prob = model.predict_proba(X_test)[:,1]
auc = roc_auc_score(y_test, y_pred_prob)
print("AUC:", auc)

In [ ]:
# Risk Forecasting / Market Risk
import numpy as np
returns = np.log(prices / prices.shift(1)).dropna()
lambda_ = 0.94
ewma_var = [0]*len(returns)
sigma2 = returns.iloc[0]**2
for t in range(1, len(returns)):
    sigma2 = lambda_ * sigma2 + (1-lambda_)*returns.iloc[t-1]**2
    ewma_var[t] = -np.sqrt(sigma2) * 1.645  # 95% VaR

In [ ]:
# Quantitative Signal Research / Factor Analysis
momentum = prices.pct_change(60)
top_quantile = 0.2
selected = momentum.iloc[-1].nlargest(int(top_quantile*len(momentum.columns))).index
daily_returns = prices.pct_change().loc[selected].mean(axis=1)
nav = (1 + daily_returns).cumprod()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 假设 prices 是 DataFrame，每列为股票收盘价
# prices = pd.read_csv("prices.csv", parse_dates=["date"]).pivot("date","ticker","adj_close")

# 1️⃣ 计算 60 天动量信号
momentum = prices.pct_change(60)

# 2️⃣ 信号分组：top 20% 和 bottom 20%
top_quantile = 0.2
bottom_quantile = 0.2
n_top = int(top_quantile * len(momentum.columns))
n_bottom = int(bottom_quantile * len(momentum.columns))

latest_signal = momentum.iloc[-1]
top_group = latest_signal.nlargest(n_top).index
bottom_group = latest_signal.nsmallest(n_bottom).index

# 3️⃣ 未来 20 天收益
future_returns = prices.pct_change(20).iloc[-20:]

top_returns = future_returns[top_group].mean(axis=1)
bottom_returns = future_returns[bottom_group].mean(axis=1)

# 4️⃣ 累积净值 (NAV)
nav_top = (1 + top_returns).cumprod()
nav_bottom = (1 + bottom_returns).cumprod()

# 5️⃣ 绘制 NAV 曲线
plt.figure(figsize=(10,6))
plt.plot(nav_top, label="Top 20% Momentum", color='green')
plt.plot(nav_bottom, label="Bottom 20% Momentum", color='red')
plt.title("NAV Curve of Top/Bottom Momentum Groups")
plt.xlabel("Date")
plt.ylabel("Cumulative NAV")
plt.legend()
plt.grid(True)
plt.show()
# NAV = 赚钱能力 + 稳定性 + 信号预测能力的综合体现

In [ ]:
#t_stat 表示两组均值差异的大小
#p_value < 0.05 → 差异显著 → 动量信号有效
#这是一种简单可解释的信号显著性检验


import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# 取最新信号
signal = momentum.iloc[-1]

# 分组
top_quantile = 0.2
bottom_quantile = 0.2
n_top = int(top_quantile * len(signal))
n_bottom = int(bottom_quantile * len(signal))

top_group = signal.nlargest(n_top).index
bottom_group = signal.nsmallest(n_bottom).index

# 未来 20 天收益
future_returns = prices.pct_change(20).iloc[-20:]

# 计算每组平均未来收益
top_returns = future_returns[top_group].mean(axis=1)
bottom_returns = future_returns[bottom_group].mean(axis=1)

# t-test
t_stat, p_value = ttest_ind(top_returns, bottom_returns)
print(f"t-statistic: {t_stat:.3f}, p-value: {p_value:.3f}")

if p_value < 0.05:
    print("Top/Bottom group difference is statistically significant.")
else:
    print("No significant difference between groups.")


In [ ]:
#coef 中的 signal 对应 𝛽
#P>|t| < 0.05 → 信号与未来收益关系显著
#可以同时检查 截距 α 是否显著（策略是否有额外收益）


import statsmodels.api as sm

# 计算未来 20 天平均收益
future_avg_return = future_returns.mean(axis=1)

# 回归
X = signal[top_group.union(bottom_group)]  # 用 top/bottom 股票
Y = future_avg_return.loc[X.index]

X = sm.add_constant(X)  # 添加截距
model = sm.OLS(Y, X).fit()
print(model.summary())


In [ ]:
## 用机器学习预测股票短期涨跌（分类策略）
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# 假设 prices 是 DataFrame，每列为股票收盘价
returns_5d = prices.pct_change(5).shift(-5)  # 未来5天收益
labels = (returns_5d > 0).astype(int)

# 构建特征：过去 5、10、20 日动量
features = pd.DataFrame()
features['mom_5'] = prices.pct_change(5)
features['mom_10'] = prices.pct_change(10)
features['mom_20'] = prices.pct_change(20)

# 对齐标签
features = features.iloc[:-5]
labels = labels.iloc[:-5]

# 只用单只股票举例
X = features['AAPL'].dropna().to_frame()
y = labels['AAPL'].loc[X.index]

# 数据拆分
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# 训练模型
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 预测
y_pred = model.predict(X_test)

# 评估
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

# 将预测信号转化为组合策略
signal = pd.Series(y_pred, index=y_test.index)
daily_returns = prices['AAPL'].pct_change().loc[signal.index]
strategy_returns = daily_returns * (signal*2 - 1)  # signal=1 -> long, 0 -> short
nav = (1 + strategy_returns).cumprod()
nav.plot(title="ML-Based Strategy NAV")



In [ ]:
# Comparative Analysis of Factor-Based and Machine Learning Quantitative Strategies

import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt

# -----------------------
# 1. Load historical prices
# -----------------------
# prices = pd.read_csv("prices.csv", parse_dates=["date"]).pivot("date","ticker","adj_close")
returns = prices.pct_change().dropna()

# -----------------------
# 2. Factor-based Strategy: Momentum
# -----------------------
momentum = prices.pct_change(60)
top_quantile = 0.2
rebalance_dates = returns.resample("ME").last().index
portfolio_returns_factor = []
current_weights = pd.Series(0.0, index=prices.columns)

for i in range(1, len(rebalance_dates)):
    start, end = rebalance_dates[i-1], rebalance_dates[i]
    signal_date = momentum.index.asof(start)
    signal = momentum.loc[signal_date].dropna()
    cutoff = signal.quantile(1-top_quantile)
    selected = signal[signal >= cutoff].index
    new_weights = pd.Series(0.0, index=prices.columns)
    new_weights[selected] = 1.0 / len(selected)
    # daily returns
    period_returns = returns.loc[start:end].iloc[1:]
    daily_port_ret = period_returns.dot(new_weights)
    portfolio_returns_factor.append(daily_port_ret)
    current_weights = new_weights

portfolio_returns_factor = pd.concat(portfolio_returns_factor)
nav_factor = (1 + portfolio_returns_factor).cumprod()

# -----------------------
# 3. ML-based Strategy: Random Forest Classification
# -----------------------
# Target: next 5-day return sign
future_returns = prices.pct_change(5).shift(-5)
labels = (future_returns > 0).astype(int)

# Features: past 5/10/20 day returns
features = pd.DataFrame()
features['mom_5'] = prices.pct_change(5)
features['mom_10'] = prices.pct_change(10)
features['mom_20'] = prices.pct_change(20)

# Align features and labels
features = features.iloc[:-5]
labels = labels.iloc[:-5]

# Only single stock example, can expand to all
X = features['AAPL'].dropna().to_frame()
y = labels['AAPL'].loc[X.index]

train_size = int(len(X)*0.7)
X_train, X_test = X.iloc[:train_size], X.iloc[train_size:]
y_train, y_test = y.iloc[:train_size], y.iloc[train_size:]

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
signal_pred = model.predict(X_test)

# Convert prediction to returns
daily_returns = prices['AAPL'].pct_change().loc[signal_pred.index]
strategy_returns_ml = daily_returns * (signal_pred*2 - 1)
nav_ml = (1 + strategy_returns_ml).cumprod()

# -----------------------
# 4. Compare NAVs
# -----------------------
plt.figure(figsize=(10,6))
plt.plot(nav_factor, label="Factor-Based Momentum", color='green')
plt.plot(nav_ml, label="ML-Based Strategy", color='blue')
plt.title("NAV Comparison: Factor vs ML Strategy")
plt.xlabel("Date")
plt.ylabel("Cumulative NAV")
plt.legend()
plt.grid(True)
plt.show()

